# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In this second notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078). This model will achieve improved test perplexity whilst only using a single layer RNN in both the encoder and the decoder.

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](assets/seq2seq1.png)

We use our encoder (green) over the embedded source sequence (yellow) to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.

In the previous model, we used an multi-layered LSTM as the encoder and decoder.

![](assets/seq2seq4.png)

One downside of the previous model is that the decoder is trying to cram lots of information into the hidden states. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded so far. By alleviating some of this information compression, we can create a better model!

We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory). Why? Mainly because that's what they did in the paper (this paper also introduced GRUs) and also because we used LSTMs last time. To understand how GRUs (and LSTMs) differ from standard RNNS, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) link. Is a GRU better than an LSTM? [Research](https://arxiv.org/abs/1412.3555) has shown they're pretty much the same, and both are better than standard RNNs. 

## Install required libraries

We'll be coding up the models in PyTorch and using torchtext to help us do all of the pre-processing required. We'll also be using spaCy to assist in the tokenization of the data. Install spaCy tokenizers for English and German. 

In [ ]:
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!wget "http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz"
!wget "http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz"
!wget "http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/mmt16_task1_test.tar.gz"

## Imports

Import evertyhing we need to train the model.

In [2]:
from typing import List, Iterable

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k

import spacy
import numpy as np

import random
import math
import time

We'll set the random seeds for deterministic results.

In [3]:
SEED: int = 1234

def random_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.determenistic = True

random_seed(SEED)

The dataset we'll be using is the [Multi30k dataset](https://github.com/multi30k/dataset). This is a dataset with ~30,000 parallel English, German and French sentences, each with ~12 words per sentence. Currently, the URL for multi30k is broken so we have to replace with temporary links to download dataset. 

Next, we'll create the tokenizers. A tokenizer is used to turn a string containing a sentence into a list of individual tokens that make up that string, e.g. "good morning!" becomes ["good", "morning", "!"]. We'll start talking about the sentences being a sequence of tokens from now, instead of saying they're a sequence of words. What's the difference? Well, "good" and "morning" are both words and tokens, but "!" is a token, not a word. 

```
tokenizer_de = get_tokenizer("spacy", language="de_core_news_sm")
tokenizer_en = get_tokenizer("spacy", language="en_core_web_sm")
```
spaCy has model for each language ("de_core_news_sm" for German and "en_core_web_sm" for English) which need to be loaded so we can access the tokenizer of each model. 

In [4]:
SOURCE_LANGUAGE = 'de'
TARGET_LANGUAGE = 'en'

token_transform = {}
vocab_transform = {}

token_transform[SOURCE_LANGUAGE] = get_tokenizer("spacy", language="de_core_news_sm")
token_transform[TARGET_LANGUAGE] = get_tokenizer("spacy", language="en_core_web_sm")


def yield_tokens(data_iter: Iterable, language: str):
    language_index = {SOURCE_LANGUAGE: 0, TARGET_LANGUAGE: 1}
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

Next, we'll build the *vocabulary* for the source and target languages. The vocabulary is used to associate each unique token with an index (an integer). The vocabularies of the source and target languages are distinct.

`torchtext`'s `build_vocab_from_iterator` would handle create of vocabulary for us. We have to set a yield fucntion for data processing where we will tokenize sentences. Add special tokens with their correspoding indexes with in the vocabulary.Using the `min_freq` argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. Tokens that appear only once are converted into an `<unk>` (unknown) token. Special tokens: `<unk>` - unknown token which is not in the vocabulary, `<pad>` - padding token to make senteces equal lenght for dataloader, `<bos>` - begining of the sentence, `<eos>` - end of the sentence.   

It is important to note that our vocabulary should only be built from the training set and not the validation/test set. This prevents "information leakage" into our model, giving us artifically inflated validation/test scores.

In [5]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [6]:
for language in [SOURCE_LANGUAGE, TARGET_LANGUAGE]:
    train_iter = Multi30k(root="/content", split="train", language_pair=(SOURCE_LANGUAGE, TARGET_LANGUAGE))
    vocab_transform[language] = build_vocab_from_iterator(
        yield_tokens(train_iter, language),
        min_freq=2,
        specials=special_symbols,
        special_first=True
    )

for language in [SOURCE_LANGUAGE, TARGET_LANGUAGE]:
    vocab_transform[language].set_default_index(UNK_IDX)

We also need to define a `torch.device`. This is used to tell torchText to put the tensors on the GPU or not. We use the `torch.cuda.is_available()` function, which will return `True` if a GPU is detected on our computer. We pass this `device` to the iterator.


In [7]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Unlike previous version, `torchtext Field` did for us  tokenization and added special symbols, with current version of `torchtext>=0.12.0` you have to make a pipeline for text processing. `text_transform` is dictionary containg all necessary tranformation for source and target languages. Firsly, we tokenize sentences. Secondly, transform it into vocavulary indexes and add special symbol `<bos>` and `<eos>` for indication of start of senteces and ends. Finaly, transform it into `torch.Tensor`.

In [8]:
def sequential_transforms(*transforms):
    def func(text_input: str):
        for transform in transforms:
            text_input = transform(text_input)
        return text_input
    return func

def to_tensor(token_ids: List[int]):
    return torch.cat((
        torch.tensor([BOS_IDX]),
        torch.tensor(token_ids),
        torch.tensor([EOS_IDX]),
    ))

text_transform = {}

for language in [SOURCE_LANGUAGE, TARGET_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language],  # Tokenization
        vocab_transform[language],  # Numericalization
        to_tensor  # Add BOS/EOS and create tensor
    )

def collate_fn(batch):
    source_batch, target_batch = [], []
    for source_sample, target_sample in batch:
        source_batch.append(text_transform[SOURCE_LANGUAGE](source_sample.rstrip("\n")))
        target_batch.append(text_transform[TARGET_LANGUAGE](target_sample.rstrip("\n")))

    source_batch = pad_sequence(source_batch, batch_first=False, padding_value=PAD_IDX)
    target_batch = pad_sequence(target_batch, batch_first=False, padding_value=PAD_IDX)
    return {
        "input": source_batch,
        "target": target_batch,
    }

Select a batch size for our dataloaders and make `collate_fn` function to pad our sequences. When we get a batch of examples using an iterator we need to make sure that all of the source sentences are padded to the same length, the same with the target sentences. Luckily, torchText iterators handle this for us! We use a `pad_sequence` to creates batches.


In [9]:
BATCH_SIZE = 64

train_iter = Multi30k(root="/content", split='train', language_pair=(SOURCE_LANGUAGE, TARGET_LANGUAGE))
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

valid_iter = Multi30k(root="/content", split='valid', language_pair=(SOURCE_LANGUAGE, TARGET_LANGUAGE))
valid_dataloader = DataLoader(valid_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

## Building the Seq2Seq Model

### Encoder

The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

The rest of the encoder should be very familar from the last tutorial, it takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, passes it through the embedding layer, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

This is identical to the encoder of the general seq2seq model, with all the "magic" happening inside the GRU (green).

![](assets/seq2seq5.png)

In [10]:
class Encoder(nn.Module):
    def __init__(
        self,
        input_dim: int,
        embed_dim: int,
        hidden_dim: int,
        dropout: float
    ):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        embeddings = self.embedding(x)
        
        outputs, hidden = self.rnn(embeddings)

        return hidden

## Decoder

The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

Instead of the GRU in the decoder taking just the embedded target token, $d(y_t)$ and the previous hidden state $s_{t-1}$ as inputs, it also takes the context vector $z$. 

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Note how this context vector, $z$, does not have a $t$ subscript, meaning we re-use the same context vector returned by the encoder for every time-step in the decoder. 

Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Thus, our decoder now looks something like this:

![](assets/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.

How do these two changes reduce the information compression? Well, hypothetically the decoder hidden states, $s_t$, no longer need to contain information about the source sequence as it is always available as an input. Thus, it only needs to contain information about what tokens it has generated so far. The addition of $y_t$ to the linear layer also means this layer can directly see what the token is, without having to get this information from the hidden state. 

However, this hypothesis is just a hypothesis, it is impossible to determine how the model actually uses the information provided to it (don't listen to anyone that says differently). Nevertheless, it is a solid intuition and the results seem to indicate that this modifications are a good idea!

Within the implementation, we will pass $d(y_t)$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $d(y_t), s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. We also don't pass a value of dropout to the GRU as it only uses a single layer.

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $d(y_t)$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [11]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim: int,
        embed_dim: int,
        hidden_dim: int,
        dropout: float
    ):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim, embed_dim)

        self.rnn = nn.GRU(embed_dim + hidden_dim, hidden_dim)

        self.fc = nn.Linear(embed_dim + hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]

        input  = input.unsqueeze(0)
        #input = [1, batch size]

        embeddings = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]

        emb_con = torch.cat((embeddings, context), dim=2)
        #emb_con = [1, batch size, emb dim + hid dim]

        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]

        output = torch.cat(
            (embeddings.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1
        )

        prediction = self.fc(output)

        return prediction, hidden

## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](assets/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [36]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, device: str):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hidden_dim == decoder.hidden_dim, (
            "Hidden dimensions of encoder and decoder must be equal!"
        )

    def forward(self, data, teacher_forcing_ratio: float = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        for k, v in data.items():
            data[k] = v.to(self.device)

        batch_size = data["target"].size(1)
        target_len = data["target"].size(0)
        target_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)

        #last hidden state of the encoder is the context
        context = self.encoder(data["input"])

        #context also used as the initial hidden state of the decoder
        hidden = context

        input = data["target"][0, :]

        for t in range(1, target_len):

            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)

            input = data["target"][t, :] if teacher_force else top1

        return outputs

# Training the Seq2Seq Model

The rest of this tutorial is very similar to the previous one. 

We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [37]:
INPUT_DIM = len(vocab_transform[SOURCE_LANGUAGE])
OUTPUT_DIM = len(vocab_transform[TARGET_LANGUAGE])
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [38]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0., std=0.01)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8014, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6191, 256)
    (rnn): GRU(768, 512)
    (fc): Linear(in_features=1280, out_features=6191, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,719,023 trainable parameters


We initiaize our optimizer.

In [40]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [41]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

We then create the training loop...

In [42]:
def train(model, dataloader, optimizer, criterion, clip: float):
    model.train()

    epoch_loss = 0

    for i, data in enumerate(dataloader):

        optimizer.zero_grad()

        output = model(data)

        output_dim = output.size(-1)
        # print(data["target"].shape, output.shape)
        # torch.Size([24, 64]) torch.Size([24, 64, 6191])
        output = output[1:].view(-1, output_dim)
        target = data["target"][1:].view(-1)
        
        loss = criterion(output, target)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(list(dataloader))

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [46]:
def evaluate(model, dataloader, optimizer, criterion):
    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for i, data in enumerate(dataloader):

            optimizer.zero_grad()

            output = model(data, 0)

            output_dim = output.size(-1)

            output = output[1:].view(-1, output_dim)
            target = data["target"][1:].view(-1)
            
            loss = criterion(output, target)

            epoch_loss += loss.item()

    return epoch_loss / len(list(dataloader))

We'll also define the function that calculates how long an epoch takes.

In [47]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [48]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float("inf")

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, optimizer, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pth')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 52s
	Train Loss: 4.134 | Train PPL:  62.440
	 Val. Loss: 4.634 |  Val. PPL: 102.923
Epoch: 02 | Time: 0m 51s
	Train Loss: 3.642 | Train PPL:  38.175
	 Val. Loss: 4.210 |  Val. PPL:  67.377
Epoch: 03 | Time: 0m 53s
	Train Loss: 3.175 | Train PPL:  23.928
	 Val. Loss: 3.961 |  Val. PPL:  52.489
Epoch: 04 | Time: 0m 52s
	Train Loss: 2.789 | Train PPL:  16.260
	 Val. Loss: 3.892 |  Val. PPL:  49.004
Epoch: 05 | Time: 0m 52s
	Train Loss: 2.477 | Train PPL:  11.911
	 Val. Loss: 3.823 |  Val. PPL:  45.740
Epoch: 06 | Time: 0m 51s
	Train Loss: 2.190 | Train PPL:   8.937
	 Val. Loss: 3.852 |  Val. PPL:  47.078
Epoch: 07 | Time: 0m 51s
	Train Loss: 1.960 | Train PPL:   7.101
	 Val. Loss: 3.806 |  Val. PPL:  44.953
Epoch: 08 | Time: 0m 51s
	Train Loss: 1.765 | Train PPL:   5.842
	 Val. Loss: 3.886 |  Val. PPL:  48.721
Epoch: 09 | Time: 0m 52s
	Train Loss: 1.605 | Train PPL:   4.976
	 Val. Loss: 3.953 |  Val. PPL:  52.089
Epoch: 10 | Time: 0m 52s
	Train Loss: 1.461 | Train PPL

Finally, we test the model on the test set using these "best" parameters.

In [49]:
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [60]:
train_iter = Multi30k(root="/content", split='test', language_pair=(SOURCE_LANGUAGE, TARGET_LANGUAGE))
test_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

test_loss = evaluate(model, test_dataloader, optimizer, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.778 | Test PPL:  43.742 |


In [50]:
model.eval()

epoch_loss = 0
with torch.no_grad():
    for i, batch in enumerate(valid_dataloader):
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)

        output = model(batch, 0)
        break

In [51]:
idx2word = vocab_transform[TARGET_LANGUAGE].get_itos()

In [52]:
def reverse(index):
    string = []
    for idx in index:
        if idx > 3:
            string.append(idx2word[idx])
    return " ".join(string)

In [53]:
ground_truths = []
for i in range(64):
    gt = batch["target"][:, i]
    ground_truths.append(reverse(gt.detach().cpu().numpy()))

In [54]:
predictions = []
for i in range(64):
    test_output = output[:, i].argmax(-1)
    predictions.append(reverse(test_output.detach().cpu().numpy()))


In [55]:
for ground_truth, prediction in zip(ground_truths, predictions):
    print(ground_truth)
    print(prediction)
    print()

A group of men are loading cotton onto a truck
A group of men are a a roller - truck .

A man sleeping in a green room on a couch .
A man sleeping sleeping in a couch in a couch .

A boy wearing headphones sits on a woman 's shoulders .
A boy sitting on his lap sitting on a lap .

Two men setting up a blue ice fishing hut on an iced over lake
Two men are on a green boat on a rocky shore .

A balding man wearing a red life jacket is sitting in a small boat .
A bald man with a a red swim trunks sitting in a pool . .

A lady in a red coat , holding a hand bag likely of asian descent , jumping off the ground for a .
A woman in a red coat coat is taking a a into a air to a a a into the distance . . .

A brown dog is running after the black dog .
A brown dog runs running the brown dog . .

A young boy wearing a jersey swings a baseball bat at an incoming pitch .
A little boy with a baseball bat in a bat bat to bat a bat . .

A man in a cluttered office is using the telephone
A man is talking